In [1]:
using JLD2
using Printf

In [5]:
function extract_from_config_table(config_filepath, key, run_idx)
    config_file = open(config_filepath, "r")
    header = readline(config_file)
    col_idx = findfirst(item -> cmp(item, key) == 0, split(header, " "))
    for line=readlines(config_file)
        entries = split(line, " ")
        if parse(Int, entries[1]) == run_idx
            close(config_file)
            return parse(Float64, entries[col_idx])
        end
    end
    close(config_file)
    return -1
end

function generate_table_data(buffer, datum; format_string="%s", header=false)
    elem_type = header ? "th" : "td"

    format = Printf.Format(format_string)
    print(buffer, @sprintf("<%s>", elem_type))
    print(buffer, Printf.format(format, datum))
    print(buffer, @sprintf("</%s>", elem_type))
end

function generate_table_row(buffer, data_list; format_strings=nothing, header=false)
    print(buffer, "<tr>")
    for (i, data)=enumerate(data_list)
        if !isnothing(format_strings)
            generate_table_data(buffer, data; format_string=format_strings[i], header)
        else
            generate_table_data(buffer, data; header)
        end
    end
    print(buffer, "</tr>\n")
end

function generate_detail_row(buffer, names, values; format_strings=nothing)
    print(buffer, "<tr><td colspan=\"7\"><details>")
    for (i, data)=enumerate(names)
        format = Printf.Format(format_strings[i])
        print(buffer, @sprintf("<b>%s</b>: ", data))
        print(buffer, Printf.format(format, values[i]))
        print(buffer, "<br>")
    end
    print(buffer, "</details></td></tr>\n")
end

function extract_from_jld_file(file, key)
    return file[key]
end

function find_in_param_file(param_text, key, config_file, run_idx)
    regex = Regex("^\\s*$key\\s*=\\s*(?<value>\\S*).*\$", "m")
    m=match(regex, param_text)
    if occursin("parse", m[:value])
        return extract_from_config_table(config_file, key, run_idx)
    else
        return parse(Float64, m[:value])
    end
end

function extract_README(run_directory)
    README_file = @sprintf("%s/README.txt", run_directory)
    return strip(read(README_file, String))
end

function extract_SLURM_ID(run_directory)
    regex = r"/([0-9]+(/[0-9]+)?)/?$"
    m=match(regex, run_directory)
    return replace(m[1], "/" => "_")
end

function extract_run_idx(run_directory)
    regex = r"/[0-9]+/([0-9]+)/?$"
    m=match(regex, run_directory)
    if m !== nothing
        return parse(Int, m[1])
    else
        return 0
    end
end

function mean(series)
    return sum(series) / length(series)
end

function extract_rsw_data(ic_file, param_file, run_directory, plot_data_file, idx)

    run_idx = extract_run_idx(run_directory)
    config_file = @sprintf("%s/froude-parameters.txt", run_directory)
    param_text = read(param_file, String)

    main_table = [
        ("Run",           "<b>%d</b>",     (idx) -> idx),
        ("SLURM_ID",      "%s",     (idx) -> extract_SLURM_ID(run_directory)),
        ("nx",            "%d"    , (idx) -> extract_from_jld_file(ic_file, "grid/nx")),
        ("Spin-up time/f",  "%0.1f",  (idx) -> find_in_param_file(param_text, "spinup_T", config_file, run_idx) / extract_from_jld_file(ic_file, "params/f")),
        ("Sim. time/f",     "%0.1f",  (idx) -> find_in_param_file(param_text, "T", config_file, run_idx) / extract_from_jld_file(ic_file, "params/f")),
        ("README",        "%s",     (idx) -> extract_README(run_directory)),
        ("Figures",       "%s",     (idx) -> @sprintf("<a class=\"btn btn-primary\" href=\"https://cims.nyu.edu/~nad9961/raytracing/%s_figures.html\">Figures</a>", extract_SLURM_ID(run_directory))),
    
    ]
    
    extraction_table = [
        ("Lx",            "%0.0fπ", (idx) -> extract_from_jld_file(ic_file, "grid/Lx")/π),
        ("f",             "%0.2f",  (idx) -> extract_from_jld_file(ic_file, "params/f")),
        ("c",             "%0.2f",  (idx) -> sqrt(extract_from_jld_file(ic_file, "params/Cg2"))),
        ("K<sub>D</sub>", "%0.1f<br>",  (idx) -> extract_from_jld_file(ic_file, "params/f") / sqrt(extract_from_jld_file(ic_file, "params/Cg2"))),
        
        ("Fr<sub>U</sub>","%0.2e",  (idx) -> mean(extract_from_jld_file(plot_data_file, "computed/rms_u/geo")) / sqrt(extract_from_jld_file(ic_file, "params/Cg2"))),
        ("Ro<sub>&zeta;</sub>"
                        ,"%0.2e<br>",  (idx) -> mean(extract_from_jld_file(plot_data_file, "energy/series/rms_ζ")) / extract_from_jld_file(ic_file, "params/f")),
        
        ("ν",             "%0.2e",  (idx) -> extract_from_jld_file(ic_file, "params/ν")),
        ("nν",            "%d<br>",     (idx) -> extract_from_jld_file(ic_file, "params/nν")),
        ("ag",            "%0.2e",  (idx) -> find_in_param_file(param_text, "ag", config_file, run_idx)),
        ("aw",            "%0.2e<br>",  (idx) -> find_in_param_file(param_text, "aw", config_file, run_idx)),

        ("&lt;&zeta;&gt;","%0.2e",  (idx) -> mean(extract_from_jld_file(plot_data_file, "energy/series/rms_ζ"))), # temporary correction factor
        ("&lt;U<sub>g</sub>&gt;",
                          "%0.2e",  (idx) -> mean(extract_from_jld_file(plot_data_file, "computed/rms_u/geo"))),
        ("&lt;U<sub>w</sub>&gt;",
                          "%0.2e<br>",  (idx) -> mean(extract_from_jld_file(plot_data_file, "computed/rms_u/wave"))),
        
        ("Eg(0)",         "%0.2e",  (idx) -> extract_from_jld_file(plot_data_file, "initial_condition/KE/geo") + extract_from_jld_file(plot_data_file, "initial_condition/APE/geo")),
        ("Ew(0)",         "%0.2e<br>",  (idx) -> extract_from_jld_file(plot_data_file, "initial_condition/KE/wave")+ extract_from_jld_file(plot_data_file, "initial_condition/APE/wave")),
        
        ("<span style=\"text-decoration: overline;\">Eg</span>",
                          "%0.2e",  (idx) -> mean(extract_from_jld_file(plot_data_file, "energy/series/KE/geo")) + mean(extract_from_jld_file(plot_data_file, "energy/series/APE/geo"))),
        ("<span style=\"text-decoration: overline;\">Ew</span>",
                          "%0.2e<br>",  (idx) -> mean(extract_from_jld_file(plot_data_file, "energy/series/KE/wave")) + mean(extract_from_jld_file(plot_data_file, "energy/series/APE/wave"))),
        
        ("max |&zeta;|(0)","%0.2e", (idx) -> extract_from_jld_file(plot_data_file, "initial_condition/max/ζ")),
        ("max |Ug|(0)", "%0.2e", (idx) -> extract_from_jld_file(plot_data_file, "initial_condition/max/geo_U")),
        ("max |Uw|(0)", "%0.2e<br>", (idx) -> extract_from_jld_file(plot_data_file, "initial_condition/max/wave_U")),
        
        ("<span style=\"text-decoration: overline;\">max |&zeta;|</span>","%0.2e", (idx) -> mean(extract_from_jld_file(plot_data_file, "energy/series/max/ζ"))),
        ("<span style=\"text-decoration: overline;\">max |Ug|</span>", "%0.2e", (idx) -> mean(extract_from_jld_file(plot_data_file, "energy/series/max/geo_U"))),
        ("<span style=\"text-decoration: overline;\">max |Uw|</span>", "%0.2e", (idx) -> mean(extract_from_jld_file(plot_data_file, "energy/series/max/wave_U"))),
    ]
    
    names = collect(map(x   -> x[1], main_table))
    format = collect(map(x  -> x[2], main_table))
    values = collect(map(x -> x[3](idx), main_table))

    name_details = collect(map(x   -> x[1], extraction_table))
    format_details = collect(map(x  -> x[2], extraction_table))
    value_details = collect(map(x -> x[3](idx), extraction_table))
    return names, format, values, name_details, format_details, value_details
end

function generate_table!(buffer, rsw_config_file)
    is_header_generated = false
    for (_,rsw_run)=enumerate(readlines(rsw_config_file))
        println(rsw_run)
        idx = parse(Int, split(rsw_run, " ")[1])
        directory = split(rsw_run, " ")[2]
        ic_file = jldopen(@sprintf("%s/rsw.000000.jld2", directory), "r")
        param_file = @sprintf("%s/Parameters.jl", directory)
        plot_data_file = jldopen(@sprintf("%s/plot_data.jld2", directory))
        
        names, format_strings, values, name_details, format_details, value_details = extract_rsw_data(ic_file, param_file, directory, plot_data_file, idx)
        if !is_header_generated
            generate_table_row(buffer, names; header=true)
            is_header_generated = true
        end
        generate_table_row(buffer, values; format_strings)
        generate_detail_row(buffer, name_details, value_details; format_strings=format_details)
        close(ic_file)
        close(plot_data_file)
    end
    return
end

table_contents_buffer = IOBuffer()
rsw_config_file = "/home/nad9961/JuliaRaytracingSW/analysis/Notebooks/rsw_suite/rsw-runs.txt"
generate_table!(table_contents_buffer, rsw_config_file)
table_contents = String(take!(table_contents_buffer))

1 /scratch/nad9961/rsw/57233890/5/ 57233890_5
2 /scratch/nad9961/rsw/57233890/6/ 57233890_6
3 /scratch/nad9961/rsw/57233890/7/ 57233890_7
4 /scratch/nad9961/rsw/57233890/8/ 57233890_8
5 /scratch/nad9961/rsw/57233890/9/ 57233890_9
6 /scratch/nad9961/rsw/57233890/10/ 57233890_10
7 /scratch/nad9961/rsw/57233890/11/ 57233890_11
8 /scratch/nad9961/rsw/57233890/12/ 57233890_12
9 /scratch/nad9961/rsw/57467976/1/ 57467976_1
10 /scratch/nad9961/rsw/57467976/2/ 57467976_2
11 /scratch/nad9961/rsw/57467976/3/ 57467976_3
12 /scratch/nad9961/rsw/57467976/4/ 57467976_4
13 /scratch/nad9961/rsw/57467976/5/ 57467976_5
14 /scratch/nad9961/rsw/57467976/6/ 57467976_6
15 /scratch/nad9961/rsw/57478245/1/ 57478245_1
16 /scratch/nad9961/rsw/57478245/2/ 57478245_2
17 /scratch/nad9961/rsw/57478245/3/ 57478245_3
18 /scratch/nad9961/rsw/57478245/4/ 57478245_4
19 /scratch/nad9961/rsw/57478245/7/ 57478245_7
21 /vast/nad9961/rsw/59957924/1/ 59957924_1
22 /vast/nad9961/rsw/59970376/1/ 59970376_1
23 /vast/nad9961/rsw/6

"<tr><th>Run</th><th>SLURM_ID</th><th>nx</th><th>Spin-up time/f</th><th>Sim. time/f</th><th>README</th><th>Figures</th></tr>\n<tr><td><b>1</b></td><td>57233890_5</td><td>512</td><td>1272.7</td><td>1333.3</td><td>Running matching RSW simulations with SWQG raytracing</td><" ⋯ 107276 bytes ⋯ "0e-01<br><br><b><span style=\"text-decoration: overline;\">max |&zeta;|</span></b>: 4.39e+00<br><b><span style=\"text-decoration: overline;\">max |Ug|</span></b>: 9.72e-01<br><b><span style=\"text-decoration: overline;\">max |Uw|</span></b>: 7.31e-01<br></details></td></tr>\n"

In [6]:
function replace_template_content(template_file, output_buffer, var_table)
    for line=readlines(template_file)
        token_regex = r"(?:\$([a-zA-Z0-9_]+))+"
        m = match(token_regex, line)
        if m !== nothing
            for token=m.captures
                if haskey(var_table, token)
                    line = replace(line, ("\$"*token) => var_table[token])
                else
                    println("Token ", token, " not found")
                end
            end
        end
        print(output_buffer, line)
        print(output_buffer, "\n")
    end
end

output_buffer = open("raytracing/index.html", "w")
var_table = Dict("TABLE_CONTENT" => table_contents)
replace_template_content("/home/nad9961/JuliaRaytracingSW/visualization/index_template.html", output_buffer, var_table)
close(output_buffer)

In [7]:
function generate_image_pages!(rsw_config_file)
    for (_, rsw_run)=enumerate(readlines(rsw_config_file))
        idx = parse(Int, split(rsw_run, " ")[1])
        run_directory = split(rsw_run, " ")[2]
        slurm_id = extract_SLURM_ID(run_directory)
        ic_file = jldopen(@sprintf("%s/rsw.000000.jld2", run_directory), "r")
        plot_data_file = jldopen(@sprintf("%s/plot_data.jld2", run_directory))
        output_buffer = open(@sprintf("raytracing/%s_figures.html", slurm_id), "w")
        
        var_table = Dict(
            "RUN_ID"     => @sprintf("%s", slurm_id),
            "GRID_DIM"   => @sprintf("%d", extract_from_jld_file(ic_file, "grid/nx")),

            "ROSSBY_VALUE"   => @sprintf("%0.1e", mean(extract_from_jld_file(plot_data_file, "energy/series/rms_ζ")) / extract_from_jld_file(ic_file, "params/f")),
            "FROUDE_VALUE"   => @sprintf("%0.1e", mean(extract_from_jld_file(plot_data_file, "computed/rms_u/geo")) / sqrt(extract_from_jld_file(ic_file, "params/Cg2"))),
            "GEO_VALUE"   => @sprintf("%0.1e", extract_from_jld_file(plot_data_file, "initial_condition/KE/geo") + extract_from_jld_file(plot_data_file, "initial_condition/APE/geo")),
            "WAVE_VALUE"   => @sprintf("%0.1e", extract_from_jld_file(plot_data_file, "initial_condition/KE/wave") + extract_from_jld_file(plot_data_file, "initial_condition/APE/wave")),
                
            "DIAG_TOTAL" => @sprintf("images/%s_energy_total.png", slurm_id),
            "DIAG_KE_PE" => @sprintf("images/%s_energy_KE_PE.png", slurm_id),
            "DIAG_GEO"   => @sprintf("images/%s_energy_geo.png", slurm_id),
            "DIAG_WAVE"  => @sprintf("images/%s_energy_wave.png", slurm_id),

            "SNAP_1_Q" => @sprintf("images/%s_start_pv_snapshot.png", slurm_id),
            "SNAP_1_DIV" => @sprintf("images/%s_start_divergence_snapshot.png", slurm_id),
            "SNAP_2_Q"   => @sprintf("images/%s_final_pv_snapshot.png", slurm_id),
            "SNAP_2_DIV"  => @sprintf("images/%s_final_divergence_snapshot.png", slurm_id),
            
            "RADIAL_INITIAL" => @sprintf("images/%s_energy_radial_initial.png", slurm_id),
            "RADIAL_AVERAGE" => @sprintf("images/%s_energy_radial_average.png", slurm_id),
            "RADIAL_INITIAL_KE" => @sprintf("images/%s_kinetic_energy_radial_initial.png", slurm_id),
            "RADIAL_AVERAGE_KE" => @sprintf("images/%s_kinetic_energy_radial_average.png", slurm_id),
            "RADIAL_INITIAL_PE" => @sprintf("images/%s_potential_energy_radial_initial.png", slurm_id),
            "RADIAL_AVERAGE_PE" => @sprintf("images/%s_potential_energy_radial_average.png", slurm_id),
            
            "FLUX_TOTAL"      => @sprintf("images/%s_flux_total.png", slurm_id),
            "FLUX_COMPONENTS" => @sprintf("images/%s_flux_components.png", slurm_id))

        replace_template_content("/home/nad9961/JuliaRaytracingSW/visualization/figure_template.html", output_buffer, var_table)
        close(output_buffer)
        close(ic_file)
        close(plot_data_file)
    end
end

generate_image_pages!("/home/nad9961/JuliaRaytracingSW/analysis/Notebooks/rsw_suite/rsw-runs.txt")
print("Done!")

Done!